<div align=center>
		
<p></p>
<p></p>
<font size=5>
In the Name of God
<font/>
<p></p>
 <br/>
    <br/>
    <br/>
<font color=#FF7500>
Sharif University of Technology - Departmenet of Computer Engineering
</font>
<p></p>
<font color=blue>
Artifical Intelligence - Dr. Mohammad Hossein Rohban
</font>
<br/>
<br/>
Fall 2021

</div>

<hr/>
		<div align=center>
		    <font color=red size=6>
			    <br />
Practical Assignment 5-Q1
            	<br/>
			</font>
    <br/>
    <br/>
<font size=4>
			<br/><br/>
Deadline:  Bahman 17th
                <br/><b>
              Cheating is Strongly Prohibited
                </b><br/><br/>
                <font color=red>
Please run all the cells.
     </font>
</font>
                <br/>
    </div>

# Personal Data

In [1]:
# Set your student number
student_number = 98100118
Name = 'parham'
Last_Name = 'chavoshian'

# Rules
- You are not allowed to use provided codes that can be found on the internet. 
- If you want to use a library which is not already imported, you must ask a question on Quera to get the permission of using that.
- Do not hesitate to ask questions on Quera, if you have any.
- This assignment is due Bahman 17th 23:59:59. you can use up to 1 grace day for this assignment and the hard deadline is Bahman 18th 23:59:59.

# Q1 (30Points)

## Mountain Car

The OpenAI Gym library includes a set of Python Reinforcement Learning environments. In this question, we examine the Mountain Car environment of this collection. Mountain Car is a Reinforcement Learning task that aims to learn the policy of climbing a steep hill and reaching the flag-marked goal. Also, the car engine is not powerful enough to climb straight up the hill on the right. Therefore, it must gain enough acceleration by climbing the hill on the left.


In this case, the state of the car is determined by an array containing its position and speed.

| Num | Observation | Min | Max|
| --- | --- | --- | ---|
| 0 | Position | -1.3 | 0.5 |
| 1 | Velocity | -0.07 | 0.07|

The intelligent agent is allowed to perform three movements of right push, no push and left push in each step. The agent's move will be given to the environment and the environment returns the next state along with the movement reward. For each step that the car does not reach the target, a cost of -1 is considered. Use Q-learning to find the optimal policy in each case. To do this, you must complete the functions below.

** Note that you will receive the full score of this question only if you can achieve a score of -115 with your implementation.

In [13]:
import numpy as np
import random
import gym
from tqdm import tqdm
from tqdm import trange

In [14]:
env = gym.make("MountainCar-v0")
# initialization (parameters such as learning rate, gamma, number of states, number of actions, and q_table)

env.reset()
pos_low, pos_high = -1.2, 0.6
vel_low, vel_high =  -0.07, 0.07
num_states = np.array([(0.6 + 1.2) * 10, (0.07 + 0.07) * 100])
num_states = np.round(num_states, 0).astype(np.int64) + 1
q_table = np.random.uniform(low = -1, high = 1, size = (num_states[0], num_states[1], 3))
q_table.shape

(19, 15, 3)

Next cell wants you supplement two functions. First for transforming the continuous space into discrete one (in order to make using q_table feasible), second for updating q_values based on the last action done by agent.

In [15]:
def discretize_state(x, minn, factor):
    return int((x - minn) * factor)


def env_state_to_Q_state(state):
    [position, velocity] = state
    pos_low, pos_high = -1.2, 0.6
    vel_low, vel_high =  -0.07, 0.07
    disc_state = np.array([discretize_state(position, pos_low, 10), discretize_state(velocity, vel_low, 100)])
    return disc_state

def update_q(current_state, next_state, action, reward, lr, discount):
    temp = lr * (reward + discount * np.max(q_table[next_state[0], next_state[1]]) - q_table[current_state[0], current_state[1], action])
    q_table[current_state[0], current_state[1], action] += temp

At the following cell, the ends of two functions are getting current action based on the policy and defining the training process respectively.

In [16]:
def get_action(current_state):
    return np.argmax(q_table[current_state[0], current_state[1]])


def q_learning():
    iter_num = 50000
    epsilon = 0.8
    lr = 0.1
    discount = 0.9
    min_epsilon = 0
    reduct_epsilon = (epsilon - min_epsilon) / iter_num
    reward_list = []
    avg_reward_list = []
    t = trange(iter_num, desc='', leave=True)
    for i in t:
        reward, total_reward = 0, 0
        curr_state = env.reset()
        curr_state = env_state_to_Q_state(curr_state)
        done = False
        while not(done):
            rand_num = np.random.random()
            if rand_num < epsilon:
                action = np.random.randint(0, 3)
            else:
                action = get_action(curr_state)
            next_state, reward, done, info = env.step(action) 
            next_state = env_state_to_Q_state(next_state)
            if done and next_state[0] >= 0.5:
                q_table[next_state[0], next_state[1], action] = reward
            else:
                update_q(curr_state, next_state, action, reward, lr, discount)
            total_reward += reward
            curr_state = next_state
        if epsilon > min_epsilon:
            epsilon -= reduct_epsilon
        reward_list.append(total_reward)
        if not((i + 1) % 100):
            avg_reward_list.append(np.mean(reward_list))
            reward_list.clear()
            t.set_description("Average Reward: {avg}".format(avg = avg_reward_list[-1]))
            t.refresh();


def save_policy():
    policy  = np.argmax(q_table, axis=2)
    np.save('policy.npy', policy)

In [17]:
q_learning()
save_policy()

Average Reward: -200.0: 100%|██████████| 50000/50000 [10:15<00:00, 81.24it/s] 


In [ ]:
# Attention: don't change this function. we will use this to grade your policy which you will hand in with policy.npy
# btw you can use it to see how you are performing. Uncomment two lines which are commented to be able to see what is happening visually.
def score():
    policy, scores = np.load("policy.npy"), []
    print(policy)
    for episode in range(1000):
        print(f"******Episode {episode}")
        state, score, done, step = env_state_to_Q_state(env.reset()), 0, False, 0
        while not done:
            # time.sleep(0.04)
            action = policy[state]
            state, reward, done, _ = env.step(action)
            state = env_state_to_Q_state(state)
            step += 1
            score += int(reward)
            # env.render()
        print(f"Score:{score}")
        scores.append(score)
    print(f"Average score over 1000 run : {np.array(scores).mean()}")

score()